In [8]:
import os
import shutil
import numpy as np
import lxml.etree
import glob
import xml.etree.ElementTree as ET

'''
对VOC(XML)格式的标注文件：
1.标签名称替换
2.复制出需要的标签的图像

'''


def print_labels(labels_num, labels_filenum):
    all_label_size = 0
    labels = set()
    [labels.add(label) for label in labels_num.keys()]
    
    labels = list(labels)
    labels.sort()
    print("label size:{}".format(len(labels)))
    [print("label:{}, num:{}, file_num:{}".format(label, labels_num[label], labels_filenum[label])) for label in labels]
    for label in labels_num:
        all_label_size += labels_num[label]
    print("all_label_size:",all_label_size)
def get_classes(xml_dir):
    labels_filenum = {}
    labels_num = {}
    
    print("-"*5,"search labels in ", xml_dir)
    files = os.listdir(xml_dir)
    for file in files:
        xml_path = os.path.join(xml_dir, file)
        if not xml_path.endswith("xml"):
            continue

        try:
            xml = lxml.etree.parse(xml_path)
        except:
            print("error load:", xml_path)

        tmp_labels = set()
        objs = xml.findall('object')
        for obj in objs:
            label = obj.find('name').text
            if label not in labels_num:
                labels_filenum[label] = 1
                labels_num[label] = 1
                tmp_labels.add(label)
            else:
                if label not in tmp_labels:
                    tmp_labels.add(label)
                    labels_filenum[label] += 1
                labels_num[label] += 1

    return labels_num, labels_filenum

def get_classes_in_paths(paths):
    labels_filenum_in_paths = {}
    labels_num_in_paths = {}
    for path in paths:
        for home, dirs, files in os.walk(path):
            files = glob.glob(home + '/*.xml')
            if len(files):
                dir = home
                labels_num, labels_filenum = get_classes(dir)
                for key in labels_num.keys():
                    if key not in labels_num_in_paths:
                        labels_filenum_in_paths[key] = labels_filenum[key]
                        labels_num_in_paths[key] = labels_num[key]
                    else:
                        labels_filenum_in_paths[key] += labels_filenum[key]
                        labels_num_in_paths[key] += labels_num[key]
    
    return labels_num_in_paths, labels_filenum_in_paths
    
s=set()
def change_label(xml_dir, label_changes):
    
    
    print("-"*5,"start change_labels from {}".format(xml_dir), "-"*5)
    files = os.listdir(xml_dir)
    for file in files:
        xml_path = os.path.join(xml_dir, file)
        if not xml_path.endswith("xml"):
            continue

        try:
            xml = lxml.etree.parse(xml_path)
        except:
            print("error load:", xml_path)

        root = xml.getroot()
        # 遍历所有object元素
        for obj in root.findall('object'):
            for label_change_key in label_changes.keys():
                s.add(obj.find('name').text)
                if obj.find('name').text == label_change_key:
                    obj.find('name').text = label_changes[label_change_key]
        xml.write(xml_path)# 保存修改后的XML文件


def delete_label(xml_dir):
    label_changes = {'ywzt_yfyc':1}
    files = os.listdir(xml_dir)
    for file in files:
        if (file[-3:] == 'xml') and (file[:-4]=='biandian_03836'):
            xml_path = os.path.join(xml_dir, file)
            one_object_classes = set()
            print(xml_path)
            import xml.etree.ElementTree as ET
            tree = ET.parse(xml_path)
            root = tree.getroot()

            for child in root:
                print(child.tag)
                if child.tag == 'object':
                    for node in child:
                        print("node:",node.tag)
                        if node.tag == 'name':
                            for label_change_key in label_changes.keys():
                                print("label_change_key:", label_change_key)
                                if node.text == label_change_key:
                                    root.remove(child)


            tree = ET.ElementTree(root)
            tree.write("new.xml",encoding="utf-8",short_empty_elements=True)


def select_labels_imgxml(img_dir,xml_dir, save_dir, select_labels):
    os.makedirs(save_dir, exist_ok=True)
    files = os.listdir(xml_dir)
    for file in files:
        xml_path = os.path.join(xml_dir, file)
        if not xml_path.endswith("xml"):
            continue

        try:
            xml = lxml.etree.parse(xml_path)
        except:
            print("error load:", xml_path)

        tmp_labels = set()
        objs = xml.findall('object')
        for obj in objs:
            label = obj.find('name').text
            tmp_labels.add(label)

        if select_labels.intersection(tmp_labels):
            img_path = os.path.join(img_dir,file[:-3] + 'jpg')
            shutil.copy(img_path, save_dir)
            shutil.copy(xml_path, save_dir)
            print("copy:", xml_path)


def randon_select_imgxml(img_dir,xml_dir, save_dir):
    select_num = 30
    # 构建所有文件名的列表，label从0开始编码
    filename = []
    names = os.listdir(img_dir)
    for n in names:
        if n[-3:] == 'jpg':
            filename.append(n[0:-4])

    # 打乱文件名列表
    np.random.shuffle(filename)
    select_filename = filename[:select_num + 1]
    for file in select_filename:
        img_path = os.path.join(img_dir, file + '.jpg')
        xml_path = os.path.join(xml_dir, file + '.xml')
        if os.path.exists(img_path) and os.path.exists(xml_path):
            try:
                shutil.move(img_path, save_dir)
                shutil.move(xml_path, save_dir)
            except:
                print(img_path, " already exists")


def delect_out_labels_imgxml(dir, remove_dir, delect_labels):
    os.makedirs(remove_dir, exist_ok=True)

    files = os.listdir(dir)
    for file in files:
        xml_path = os.path.join(dir, file)
        if not xml_path.endswith("xml"):
            continue

        try:
            xml = lxml.etree.parse(xml_path)
        except:
            print("error load:", xml_path)

        tmp_labels = set()
        objs = xml.findall('object')
        for obj in objs:
            label = obj.find('name').text
            tmp_labels.add(label)

        if not delect_labels.intersection(tmp_labels):
            img_path = os.path.join(dir, file.replace('xml','jpg'))
            if os.path.exists(img_path):
                shutil.move(img_path, remove_dir)
                print("move ", img_path, " to ", remove_dir)
            if os.path.exists(xml_path):
                shutil.move(xml_path, remove_dir)
                print("move ", xml_path, " to ", remove_dir)


def copy_labels_imgxml(dir, copy_dir, select_labels):
    os.makedirs(copy_dir, exist_ok=True)

    print("copy labels from ", dir)
    files = os.listdir(dir)
    for file in files:
        xml_path = os.path.join(dir, file)
        if not xml_path.endswith("xml"):
            continue

        try:
            xml = lxml.etree.parse(xml_path)
        except:
            print("error load:", xml_path)

        tmp_labels = set()
        objs = xml.findall('object')
        for obj in objs:
            label = obj.find('name').text
            tmp_labels.add(label)

        if select_labels.intersection(tmp_labels):
            img_dir = dir
            if dir[-3:] == 'xml':
                img_dir = dir.replace('xml','image')
            if dir[-3:] == 'Annotations':
                img_dir = dir.replace('Annotations','JPEGImages')
            
            img_path = os.path.join(img_dir, file.replace('xml','jpg'))
            copy_path = os.path.join(copy_dir, file.replace('xml','jpg'))
            print(copy_path)
            if os.path.exists(img_path) and (not os.path.exists(copy_path)):
                shutil.move(img_path, copy_dir)
            # #如果存在了就删除原目录的
            # if os.path.exists(copy_path):
            #     os.remove(img_path)

            copy_path = os.path.join(copy_dir, file)
            print(copy_path)
            if os.path.exists(xml_path) and (not os.path.exists(copy_path)):
                shutil.move(xml_path, copy_dir)

            # #如果存在了就删除原目录的
            # if os.path.exists(copy_path):
            #     os.remove(xml_path)
            
            



def copy_labels_imgxml_in_paths(paths, copy_dir, select_labels):
    if os.path.exists(copy_dir):
        shutil.rmtree(copy_dir)

    for path in paths:
        for home, dirs, files in os.walk(path):
            files = glob.glob(home + '/*.xml')
            if len(files):
                dir = home
                copy_labels_imgxml(dir, copy_dir, select_labels)


def clear_img_and_annotation(dir,remove_dir):
    print("***************start clear img and annotation***************")
    clear_annotation_without_objs(dir,remove_dir)
    clear_img_annotation_unpair(dir,remove_dir)  

def clear_annotation_without_objs(dir,remove_dir):
    os.makedirs(remove_dir, exist_ok=True)

    print("***************clear_annotation_without_objs form {} ***************".format(dir))
    files = os.listdir(dir)
    for file in files:
        xml_path = os.path.join(dir, file)
        if not xml_path.endswith("xml"):
            continue
        try:
            xml = lxml.etree.parse(xml_path)
        except:
            print("error load:", xml_path)

        objs = xml.findall('object')
        if len(objs)<=0:
            img_path = xml_path.replace('xml','jpg')
            if os.path.exists(img_path):
                shutil.move(img_path, remove_dir)

            shutil.move(xml_path, remove_dir)

        
def clear_img_annotation_unpair(dir,remove_dir):
    os.makedirs(remove_dir, exist_ok=True)

    print("***************clear_img_annotation_unpair from {} ***************".format(dir))
    files = os.listdir(dir)
    for file in files:
        name = file[:-4]
        file_path = os.path.join(dir, file)
        img_path = os.path.join(dir, name + '.jpg')
        xml_path = os.path.join(dir, name + '.xml')
        if os.path.exists(xml_path) and os.path.exists(img_path):
            pass
        else:
            shutil.move(file_path, remove_dir)



def indent(elem, level=1):
    print("level:",level)
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            print(1)
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            print(2)
            elem.tail = i
        for elem in elem:
            print(ET.tostring(elem, encoding='unicode'))
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            print(3)
            elem.tail = i
    else:
        if level>1 and (not elem.tail or not elem.tail.strip()):
            print(4)
            elem.tail = i
    print("elem.tail:",len(elem.tail))
def pretty_xml(element, indent, newline, level=0):  # elemnt为传进来的Elment类，参数indent用于缩进，newline用于换行
    if element:  # 判断element是否有子元素    
        if (element.text is None) or element.text.isspace():  # 如果element的text没有内容
            element.text = newline + indent * (level + 1)
        else:
            element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * (level + 1)
            # else:  # 此处两行如果把注释去掉，Element的text也会另起一行
            # element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * level
    temp = list(element)  # 将element转成list
    for subelement in temp:
        if temp.index(subelement) < (len(temp) - 1):  # 如果不是list的最后一个元素，说明下一个行是同级别元素的起始，缩进应一致
            subelement.tail = newline + indent * (level + 1)
        else:  # 如果是list的最后一个元素， 说明下一行是母元素的结束，缩进应该少一个    
            subelement.tail = newline + indent * level
        pretty_xml(subelement, indent, newline, level=level + 1)  # 对子元素进行递归操作

def add_label_in_same_position(xml_dir):
    label_add = {'ywzt_yfyc':'hxq_yfzc','ywzt_yfyc_ mh':'hxq_yfzc','ywzt_yfyc_mh':'hxq_yfzc','ywzt_yfzc':'hxq_yfzc'}
    have_labels = set(['yf','hxq_yfps','hxq_yfzc'])

    
    import xml.etree.ElementTree as ET
    files = os.listdir(xml_dir)
    for file in files:
        xml_path = os.path.join(xml_dir, file)
        if not xml_path.endswith("xml"):
            continue

        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()  
        except:
            print("error load:", xml_path)
        
        objs = tree.findall('object')

        tmp_labels = set()
        for obj in objs:
            tmp_labels.add(obj.find('name').text)
        if have_labels.intersection(tmp_labels):
            continue

        for obj in root.findall('object'):
            for label_add_key in label_add.keys():
                if obj.find('name').text == label_add_key:
                    bndbox = obj.find('bndbox')
                    xmin = bndbox.find('xmin').text
                    xmax = bndbox.find('xmax').text
                    ymin = bndbox.find('ymin').text
                    ymax = bndbox.find('ymax').text
                    
                    new_object_elem = ET.Element('object')
                    name_elem = ET.SubElement(new_object_elem, 'name')
                    name_elem.text = 'hxq_yfzc'
                    name_elem = ET.SubElement(new_object_elem, 'pose')
                    name_elem.text = 'Unspecified'
                    name_elem = ET.SubElement(new_object_elem, 'truncated')
                    name_elem.text = '0'
                    name_elem = ET.SubElement(new_object_elem, 'difficult')
                    name_elem.text = '0'
                    bndbox_elem = ET.SubElement(new_object_elem, 'bndbox')
                    xmin_elem = ET.SubElement(bndbox_elem, 'xmin')
                    xmin_elem.text = xmin
                    ymin_elem = ET.SubElement(bndbox_elem, 'ymin')
                    ymin_elem.text = ymin
                    xmax_elem = ET.SubElement(bndbox_elem, 'xmax')
                    xmax_elem.text = xmax
                    ymax_elem = ET.SubElement(bndbox_elem, 'ymax')
                    ymax_elem.text = ymax
                    # indent(new_object_elem)    #  
                    root.append(new_object_elem)
        pretty_xml(root, '\t', '\n')  # 执行美化方法
        tree.write(xml_path)# 保存修改后的XML文件


if __name__ == '__main__':

    # path1 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/train-data/'
    # path2 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/extractedfile/'
    # path3 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/pb/'
    # path4 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/onekey/'
    # path5 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/六类数据/'
    # path6 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/状态识别/'
    # path7 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/valid_dataset/'
    # path8 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/valid_dataset_0620/'
    # path9 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/电力过检/'
    # path10 ='/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/判别公司采集训练数据集/'
    # paths = [path1, path2, path3, path4, path5, path6, path7, path8, path9, path10]

    # paths = [path3]
    # get_classes_in_paths(paths)


    # 获得一个dir的classes
    # dir = '/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/20230418整理/呼吸器'
    # dir = '/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/power/old_data/valid_dataset_0620/xml'
    # labels_num, labels_filenum = get_classes(dir)
    # print_labels(labels_num, labels_filenum)

    
    #将paths所有文件夹中含有select_labels中某个label的标注及图像复制到copy_dir
    # copy_dir = '/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/20230418整理/呼吸器油封破损/'
    # select_labels = set(['hxq_gjbs','hxq_gjtps','hxq_gjzc','hxq_yfzc','yf','ywzt_yfyc','ywzt_yfyc_ mh','ywzt_yfyc_mh','ywzt_yfzc'])
    # copy_labels_imgxml_in_paths(paths, copy_dir, select_labels)
    # labels_num, labels_filenum = get_classes(copy_dir)
    # print_labels(labels_num, labels_filenum)
    


    #标签名称替换
    label_changes = {'1':"changed"}
    change_label(r'E:\panbie_data\panbie_data\panbie_test\yfjdbs', label_changes)
    print(s)
    #将dir文件夹中含有select_labels中某个label的标注及图像复制到copy_dir
    # select_labels = set(['gcc_ps'])
    # dir =r'E:\frombeijing\C-07'
    # copy_dir = r'E:\frombeijing\gcc_ps01'
    # copy_labels_imgxml(dir, copy_dir, select_labels)





    #清理标注好的文件夹中，没有xml对应的img，没有img对应的xml，没有标注物体的xml和img
    # do_clear_img_and_annotation = 0
    # if do_clear_img_and_annotation:
        # dir = '/home/linjz1/workspace/projects/dg_power/判别/空开/kk_train'
        # remove_dir = '/home/linjz1/workspace/projects/dg_power/判别/空开//tmp'
        # clear_img_and_annotation(dir,remove_dir)
        # labels_num, labels_filenum = get_classes(dir)
        # print_labels(labels_num, labels_filenum)

    #按照label_changes对应关系，将label名称变换
    # do_change_label = 0
    # if do_change_label:
        # xml_dir = '/media/linjz1/360f623c-64be-4db4-bc5c-d6527da779f6/linjz1/workspace/cls_power/20230418整理/呼吸器'
        
        # print("*"*8,"before change_labels","*"*8)
        # labels_num, labels_filenum = get_classes(xml_dir)
        # print_labels(labels_num, labels_filenum)

        # 

        # print("*"*8,"after change_labels","*"*8)
        # labels_num, labels_filenum = get_classes(xml_dir)
        # print_labels(labels_num, labels_filenum)



    



----- start change_labels from E:\panbie_data\panbie_data\panbie_test\yfjdbs -----
{'yfjd_yf', 'yfjd_jd', 'yfjd_0'}
